Jupyter notebook to write shuffling test for single unit response to direction (and probably time). 

WH 8/8/2023

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu

In [2]:
%qtconsole --style material

In [7]:
# doing without shuffle - this from fig 3 code 
pvals = []
meanDiff = []
mw_fids = [] # field IDs from MW test
for orien in ['V','H']:
    odf = df[df.Orientation==orien]
    for fname, fgroup in odf.groupby("FieldID"):
        dirs = np.unique(fgroup.CurrDir)

        dirA = fgroup[fgroup.CurrDir==dirs[0]]
        dirB = fgroup[fgroup.CurrDir==dirs[1]]
        try:
            pvals.append(mannwhitneyu(dirA.Rate, dirB.Rate).pvalue)
            meanDiff.append(abs(dirA.Rate.mean()-dirB.Rate.mean()))
            mw_fids.append(fname)
        except:
            # pvals.append(1)
            # meanDiff.append(0)
            pass

In [31]:
# unpackaged shuffle code. shuffle 1000x, compare real meanDiff to 95% of shuffled diff
nshuffle = 1000
pvals = []
meanDiff = []
sigByshuff = []
mw_fids = [] # field IDs from MW test
for orien in ['V','H']:
    odf = df[df.Orientation==orien]
    for fname, fgroup in odf.groupby("FieldID"):

        # get real difference
        dirs = np.unique(fgroup.CurrDir)
        dirA = fgroup[fgroup.CurrDir==dirs[0]]
        dirB = fgroup[fgroup.CurrDir==dirs[1]]
        shuffDiffs = []
        try:
            pvals.append(mannwhitneyu(dirA.Rate, dirB.Rate).pvalue)
            md=abs(dirA.Rate.mean()-dirB.Rate.mean())
            meanDiff.append(md)
            mw_fids.append(fname)
            for n in range(nshuffle):
                shuffDir = np.random.permutation(fgroup.CurrDir)
                dirA = fgroup[shuffDir==dirs[0]]
                dirB = fgroup[shuffDir==dirs[1]]
                smd = abs(dirA.Rate.mean()-dirB.Rate.mean())
                shuffDiffs.append(smd)
            if md > np.percentile(shuffDiffs, 95):
                sigByshuff.append(1)
            else:
                sigByshuff.append(0)
        except:
            pass